In [ ]:
!pip install -r requirements

In [43]:
from langchain.agents import create_agent
from langchain.tools import tool , ToolRuntime
import requests
from dataclasses import dataclass
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model

In [44]:
@dataclass
class Context :
    user_id : str


@dataclass
class ResponseFormat :
    summary : str
    temperature_c : float
    temperature_f : float
    humidity : float


In [71]:
@tool('get_weather',description="return weather information for a given city" , return_direct=False)
def get_weather(city:str):
    response = requests.get(f"https://wttr.in/{city}?format=j1")
    return response.json()

@tool("locate_user" , description="Look up a user's city based on the context")
def locate_user (runtime : ToolRuntime[Context]) : 
    match runtime.context.user_id : 
        case '1' : 
            return 'cairo'
        case '2' : 
            return 'paris'
        case '3' : 
            return 'london'
        case _:
            return 'unknown'

In [72]:
model = init_chat_model(
    "ollama:qwen3:4b",
    temperature = 0.3
)

checkpointer = InMemorySaver()

In [85]:
agent = create_agent(
    model=model,
    tools=[get_weather,locate_user],
    system_prompt="You are a heplful assistant .",
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

In [86]:
config = {'configurable' : {'thread_id' : 1}}

response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What is the weather like ?"}
        ]
    },
    # Arguments passed directly to agent.invoke
    config=config, 
    context=Context(user_id=""),
)

In [87]:
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

Here's a concise weather summary for **October 5, 2023**:

- **High**: 28°C (at 15:00)  
- **Low**: 18°C (at 21:00)  
- **Conditions**: Mostly sunny with clear skies throughout the day.  
- **Wind**: 0–5 km/h (light winds, mostly from the south).  
- **Humidity**: 20%–44% (lowest at 15:00, highest at 00:00).  

This forecast indicates a warm, dry day with minimal precipitation and gentle winds. 🌞


In [90]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What is the weather like ?"}
        ]
    },
    # Arguments passed directly to agent.invoke
    config=config, 
    context=Context(user_id="2"),
)
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

28


In [88]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "and is it usual ?"}
        ]
    },
    # Arguments passed directly to agent.invoke
    config=config, 
    context=Context(user_id="1"),
)
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

The weather for the day is **mostly sunny** with a high of **28°C (82°F)** and a low of **18°C (65°F)**. The average temperature is approximately **23°C (73°F)**. There is no precipitation expected, and the UV index remains low (1). The sun is visible for **10 hours** throughout the day, indicating clear skies with minimal cloud cover.


In [89]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "what is my city ?"}
        ]
    },
    # Arguments passed directly to agent.invoke
    config=config, 
    context=Context(user_id="1"),
)
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

Your city is **Cairo**.
